## Task Definition
- input:
- ouptut:

In [1]:
def load_saga_json(path):
    with open(path, "r", encoding="utf-8") as file:
        l = []
        for i in file:
           l.append(json.loads(i)) 
    return l


## RL

#### 環境設置

In [ ]:
import gym
import numpy as np
from gym import spaces
import shlex
import pandas as pd
class AuditLogEnv(gym.Env):
    def __init__(self,logFile="G16.json"):
        # 初始化環境
        ## 基礎設定
        self.logFile=logFile
        #logguardQ有設置size但很明顯我要處理的檔案大很多因此不考慮
        self.stateDim=100 #代表傳入的狀態Dimension有100維
        self.actionDim=4 #代表有四種動作
        ## 接著是載入某份log(這邊預設維G16.json，因為它檔案很小)
        self.logs=self.loadAuditLogs(logFile)#這邊載入會使用 
        if len(self.logs) == 0:#沒有成功載入任何 AuditLog的話會出現錯誤
            raise ValueError(f"No valid log entries found in {logFile}.")
        self.attackPattern=[]#這邊logguardQ給了一個輸入維度在哪種情況下可能是攻擊，但我這邊還不確定怎麼做
        self.noiseLvl=0.05 # 一個加入可能的躁點大概佔幾趴 不確定會不會用到
        ## 以下是代理每次動作後看做了哪些事可能會更新某些值
        self.ipCounts={} # 由於AuditLog也有ip相關可能會用到
        self.logIndex=0 # 看到第幾條
        self.stepCount=0 # 現在走了幾步 
        self.anomalyCount=0
        self.visitedStates=set() # 走訪過那些狀態
        self.processedLogs()=set()# 已經處理了哪些東西 作用域: 單次回合 (Single Episode)
        self.processedLogsGlobal=set() #作用域: 整個環境的生命週期 (Entire Lifetime)。
        self.featureRanges={}# 把記錄到的某些數值縮放到某個數字區間 為了提高訓練效率和穩定性。 不確定會不會用到
        self.featureWeights={}# input的每個東西的權重，不確定要怎麼分配但先放著
        
        ## 常見弱點出現在哪些東西上
        vulnName=[]#常見的有問題的AuditLog可能的特徵 但這邊什麼都沒放 才剛架起來
        attackParamsValues=[]#logGuardQ使用常見的 XSS攻擊會出現的字串特徵 跟上面很像
        attackRegEx=[]# 仿造上面 這邊則用RegEx做做看 一樣還沒放入任何東西
        # 接下來是先載入一次這次回合的(Episode)的AuditLog 並且檢查有多少是非benign
        ## 它的主要功能是在環境載入完所有日誌後，對這些日誌進行初步的分析和統計，以瞭解數據集中異常日誌的「真實分佈」
        ###這邊我還沒改好
        for _, logEntry in self.logs.iterrows():
            if self._is_anomaly(logEntry):
                self.anomaly_count += 1
        print(f"Loaded {len(self.logs)} valid log entries from {log_file}.")
        print(f"Anomaly distribution: {self.anomaly_count}/{len(self.logs)} ({self.anomaly_count/len(self.logs)*100:.1f}% anomalies)")
        self.reset()
    def loadLogs(self,logFile,dataset:str="SAGA"):
        #預設載入SAGA資料集
        #這邊是載入AuditLog的地方，要做成pandas.DataFrame
        listColumns=['relation','timestamp','label','srcNode.UUID','srcNode.Name','srcNode.Image','srcNode.Cmdline','srcNode.Type','srcNode.Pid','dstNode.UUID','dstNode.Key','dstNode.Type','dstNode.Value','dstNode.Name','dstNode.Srcaddress','dstNode.Dstaddress','dstNode.Port','dstNode.Image','dstNode.Cmdline','dstNode.Pid','dstNode','dstNode.Path']#有甚麼樣的欄位取決於現在讀哪一個資料集
        listLogs=[]
        #讀取檔案
        try:
            with open(logFile,"r",encoding='utf-8')as file:
                for line in file:
                    listLogs.append(json.loads(line))
                    """
                    這邊先沒有用到但先不刪除
                    if lens(part) >=1:
                        listLogs.append(listParts[:11])
                    else:
                        print(f"skipping malformed line:{line.strip()},except 11 fields,got{len(listParts)}")
                        """
            pd.json_normalize(listLogs)
            if not listLogs:
                print(f"Warning:no valid log entries found in {logFile}.")
        except FileNotFoundError:
            raise FileNotFoundError(f"File {logFile} not found.")
        except Exception as e:
            raise Exception({f"Error loading file {logFile}: {str(e)}"})
        # 轉換成DataFrame
        df=pd.DataFrame(listLogs,columns=listColumns)
        # 下面則是要把某些部分有固定種類的東西轉換成數值表示
        # 舉例df['status_code'] = pd.to_numeric(df['status_code'], errors='coerce')
        df['relation'] = pd.to_numeric(df['relation'], errors='coerce')
        return df
    def isAnomaly(self,logEntry):
        #對答案
        #這邊logguardQ的實作方法是給了一些很刻意的線索然後比對線索
        #我的實作方式是直接看標籤是不是benign
        #當然我會希望可以做得更好但先這樣
        return lambda logEntry: logEntry.get('label') != "benign"
    def extractFeature(self,logEntry):
        #1 擷取logEntry的各項Value
        #2 特徵標準化以及加權
        return
    def reset():
        return
    def getState(self):
        return
    def  updateNoise():
        return
    def step(self,action):
        # 一步
        # 
        return
        
    

#### 決策機設定

In [ ]:
class AuditLogInvestigator:
    def __init__(self):
        pass
class DQN:
    def __init__(self):
        pass  
    def chooseAction():
        return
    def forward():
        return
    def updateWeights():
        return
    def reportDetection():
        pass

class PPO:
    def __init__(self):
        return
    def chooseAction():
        return
    def forward():
        return
    def updateWeights():
        return
    def reportDetection():
        pass

#### 功能

In [ ]:
def evaluateStatisticalSignificance():
    return
def runSimulation(model,env,modelName,episodes,maxStep):
    return
def displayResult(model="",dictResult:dict={}):
    precision=dictResult['tp']/(dictResult['tp']+dictResult['fp'])
    recall=dictResult['tp']/(dictResult['tp']+dictResult['fn'])
    f1Score=2*(precision*recall)/(precision+recall)
    successfulSteps=[s for s in dictResult['steps'][-100:]if s<5]
    subTitle=f"""
{model} Final Result
=====================
precision:{precision}
recall:{recall}
detection:{dictResult['detection']/len(dictResult['scores'])}({dictResult['detection']}/
{len(dictResult['scores'])*100:.1f}%)
true positive:{dictResult['tp']}
false positive:{dictResult['fp']}
true negative:{dictResult['tn']}
false negative:{dictResult['fn']}
total anomalies encountered:{dictResult['anomalies']}
({dictResult['anomalies']/dictResult['tp']+dictResult['tn']+dictResult['fp']+dictResult['fn']*100:.1f}% of steps)
Mean Reward (all episodes): {np.mean(dictResult['scores']):.4f} ± {np.std(dictResult['scores']):.4f}
Mean F1-Score (all episodes): {np.mean(f1Score):.4f} ± {np.std(f1Score):.4f}
Mean Steps per episode (all episodes): {np.mean(dictResult['steps']):.1f} ± {np.std(dictResult['steps']):.1f}
Metrics for last 100 Episodes:
Mean Reward (last 100): {np.mean(dictResult['scores'][-100:]):.4f} ± {np.std(dictResult['scores'][-100:]):.4f}
Mean F1-Scores (last 100): {np.mean(f1Score[-100:]):.4f} ± {np.std(f1Score[-100:]):.4f}
Steps to Detection (last 100, successful episodes): {np.mean(successfulSteps):.1f} ± {np.std(successfulSteps):.1f}
"""if successfulSteps else "No Successful episodes"
    print(subTitle)
    return


# 主程式

In [ ]:
if __name__ == "__main__":
    #這邊第一步驟logguardQ使用了雲端硬碟儲存結果
    # 接著取樣顯示要處理的Audit log前10行
    print("\n取樣前十筆Audit Log")
    try:
        with open("","r",encoding="utf-8")as file:
            for i in file:
                if i<10:
                    print("line")
                else:
                    break
    except FileNotFoundError:
        print("Audit Log File Not Found") 
    # 設一個np的種子
    # 設定初始化的基礎環境
    env=AuditLogEnv()
    # 設定RL的決策系統
    modelAuditLogInvestigator=AuditLogInvestigator(
    )
    modelDQN=DQN()
    modelPPO=PPO()
    episodes = 200000
    # 設定Episode(還不知道怎麼搞)
    # 開始進行模擬
    # 這邊的回傳參數很大一串唷
    # score f1Score step detection tp fp tn fn anomalies cumulativeTp acitons
    # 看起來這邊跟前面環境設置結果有關
    # 看來擷取到惡意的部分應該額外弄個list 或是dictionary
    # 至於怎麼找出來我尚未想出辦法
    resultAuditLogInvestigator=runSimulation(modelAuditLogInvestigator,env,"AuditLogInvestigator",episodes=episodes,maxStep=5)
    displayResult(resultAuditLogInvestigator)


